### Tên : Huỳnh Ngọc Công Danh
### MSSV : 19521322

In [94]:
import pandas as pd
import numpy as np


In [95]:
df = pd.read_csv('/content/loan_data.csv')
df = df.dropna()
df.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y


In [96]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [97]:
df = df.drop('Loan_ID',axis=1)
df.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [98]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_encode = pd.get_dummies(X,columns=['Gender','Married','Dependents','Education','Self_Employed','Property_Area'])
y_encode = pd.DataFrame(labelencoder.fit_transform(y))
X_encode

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0
610,4106,0.0,40.0,180.0,1.0,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0
611,8072,240.0,253.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
612,7583,0.0,187.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1


In [99]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [150]:
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
acc_score = []
k = 5
kf = KFold(n_splits=k,shuffle=True)
def Score(model,X,y):
  for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = np.array(y.iloc[train_index]), np.array(y.iloc[test_index])
    # X_train_scaled = scale.fit_transform(X_train)
    # X_test_scaled = scale.fit_transform(X_test)
    model.fit(X_train,y_train.ravel())
    pred_values = model.predict(X_test)
    acc = accuracy_score(pred_values , y_test.ravel())
    acc_score.append(acc)
  return sum(acc_score)/len(acc_score)

### Parameters

In [137]:
XGB_p = {
    'booster' : ['gbtree', 'gblinear','dart'],
    'eta' : 10,
    'gamma' : [1,10,100],
    'max_depth' : 5
}
GBM_p = {
    'n_estimators' : [100,200,300],
    'learning_rate' : 10,
    'max_depth' : 10
}


In [141]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
def XGBSearch(X_encode,y_encode,p):
  max_s = 0
  r_p = []
  a = list(p.keys())
  print(p[a[3]]+1)
  for i in p[a[0]]:
    for j in range(0,p[a[1]]+1):
      for k in p[a[2]]:
        for l in range(0,p[a[3]]+1):
          # print(i,j,k,l)
          model = XGBClassifier(booster=i,eta=j/10,gamma=k,max_depth=l)
          s = Score(model,X_encode,y_encode)
          if s > max_s:
            max_s = s
            r_p = [i,j,k,l]
  return r_p
def GBMSearch(X_encode,y_encode,p):
  r_p = []
  max_s = 0
  k = list(p.keys())
  for i in p[k[0]]:
    for j in range(1,p[k[1]]+1):
      for l in range(1,p[k[2]]+1):
        model = GradientBoostingClassifier(n_estimators=i, learning_rate=j/10,max_depth=l, random_state=0)
        s = Score(model,X_encode,y_encode)
        if s > max_s:
          max_s = s
          r_p = [i,j,l]
  return r_p

      
XGP_re = XGBSearch(X_encode,y_encode,XGB_p)
GBM_re = GBMSearch(X_encode,y_encode,GBM_p)

6


### XGBoost

In [151]:
from xgboost import XGBClassifier 
print(XGP_re)
model = XGBClassifier(booster=XGP_re[0],eta=XGP_re[1],gamma=XGP_re[2],max_depth=XGP_re[3]) 
# model = XGBClassifier()
score = Score(model,X_encode,y_encode)
print(score)

['gbtree', 0, 10, 5]
0.8083333333333333


### GBM


In [152]:
from sklearn.ensemble import GradientBoostingClassifier
model1 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
score1 = Score(model1,X_encode,y_encode)
print(score1)

0.7979166666666667


In [153]:
result = pd.DataFrame([['XGBoost',score],['GBM',score1]],columns=['name','score'])
result

,name,score
0,XGBoost,0.808333
1,GBM,0.797917
